# Building solar suitablity layers for GRIDCERF

The following code was used to build the solar suitability layers for GRIDCERF. GRIDCERF does not provide the source data directly due to some license restrictions related for direct redistribution of the unaltered source data.  However, the following details the provenance associated with each source dataset and how they were processed.


## 1. Downloading the data

### 1.1 Download GRIDCERF

Download the GRIDCERF package if you have not yet done so from here:  https://doi.org/10.5281/zenodo.6601789.  Please extract GRIDCERF inside the `data` directory of this repository as the paths in this notebook are set to that expectation.


### 1.2 Download the solar potential data


#### For gridcerf_nrel_solar_csp_centralized_potential.tif

- **Title**:   Concentrating Solar Direct Normal Potential 
- **Description from Source**:   Monthly and annual average direct normal irradiance for Hawaii and the contiguous United States 
- **Source URL**:  https://ezmt.anl.gov/mapexport/energy_potential_solar_conc_v2.zip
- **Date Accessed**:  10/14/22
- **Citation**
> Perez, R. et al. A new operational model for satellite-derived irradiances: description and validation. Solar Energy 73, 307–317 (2002).


#### For gridcerf_nrel_solar_pv_centralized_potential.tif

- **Title**:    Photovoltaic Potential 
- **Description from Source**:  Monthly and annual average global horizontal irradiance for Hawaii and the contiguous United States. 
- **Source URL**:  https://ezmt.anl.gov/mapexport/energy_potential_solar_pv_v2.zip
- **Date Accessed**:  10/14/22
- **Citation**
> Perez, R. et al. A new operational model for satellite-derived irradiances: description and validation. Solar Energy 73, 307–317 (2002).


### 1.3 Import necessary Python packages

In [1]:
import os
import glob

import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
from rasterio import features


## 2. Configuration

In [ ]:
# get the parent directory path to where this notebook is currently stored
root_dir = os.path.dirname(os.getcwd())

# data directory in repository
data_dir = os.path.join(root_dir, "data")

# GRIDCERF data directory from downloaded archive
gridcerf_dir = os.path.join(data_dir, "gridcerf")

# GRIDCERF reference data directory
reference_dir = os.path.join(gridcerf_dir, "reference")

# GRIDCERF common data directory
common_dir = os.path.join(gridcerf_dir, "common")

# GRIDCERF compiled data directory
compiled_dir = os.path.join(gridcerf_dir, "compiled")

# GRIDCERF technology_specific data directory
technology_specific_dir = os.path.join(gridcerf_dir, "technology_specific")

# template land mask raster
template_raster = os.path.join(reference_dir, "gridcerf_landmask.tif")

# source solar data directory
solar_source_dir = os.path.join(gridcerf_dir, "source", "technology_specific", "nrel_solar")

# temporary output raster for processing
temp_output_raster = os.path.join(solar_source_dir, "temporary_raster.tif")

# generate a list of all common exclusion files
common_raster_list = glob.glob(os.path.join(common_dir, "*.tif"))

# data specific to each technology 
technology_data = {"csp": {"slope_percent": 5,
                           "unit_size_dc": 100000,
                           "capacity_factor_ac": 0.25,
                           "area_percentage": 0.75,
                           "input_shapefile": os.path.join(solar_source_dir, "energy_potential_solar_conc_v2", "ez_gis.energy_potential_solar_conc_v2.shp"),
                           "shapefile_field": "ann_dni"},
                   "pv":  {"slope_percent": 10,
                           "unit_size_dc": 75000,
                           "capacity_factor_ac": 0.23,
                           "area_percentage": 0.75,
                           "input_shapefile": os.path.join(solar_source_dir, "energy_potential_solar_pv_v2", "ez_gis.energy_potential_solar_pv_v2.shp"),
                           "shapefile_field": "ann_ghi"}}


## 4. Generate solar suitability rasters

### 4.1 Functions to build suitability

In [4]:
def calc_irradiance_threshold(unit_size_dc: int, 
                              capacity_factor_ac: float, 
                              area_percentage: float) -> float:
    """Calculates the irradiance necessary to produce the AC energy output expected from the given 
    unit size (kW) and AC capacity factor (%)
    
    :param unit_size:             unit size (installed DC capacity) of the solar array in kilowatts
    :type unit_size:              int
    
    :param capacity_factor_ac:    capacity factor (AC) of the solar array
    :type capacity_factor_ac:     float
    
    :param area_percentage:       assumed percentage of square site area used for solar panels
    :type area_percentage:        float
    
    
    :return:                      irradiance required in kWh_m2_day
    
    """
    
    DC_AC_RATIO = 1.3
    HOURS_PER_DAY = 24
    ACRES_PER_MW_AC = 5.8
    ACRES_PER_KM2 = 247.105
    
    kwh_day_ac_max = (unit_size_dc * HOURS_PER_DAY) / DC_AC_RATIO
    kwh_output_actual = kwh_day_ac_max * capacity_factor_ac

    irradiance = kwh_output_actual / unit_size_dc
    irradiance = round(irradiance, 3)
    
    land = ((((unit_size_dc/1000) / DC_AC_RATIO) * ACRES_PER_MW_AC) / ACRES_PER_KM2) / area_percentage
    land = round(land, 2)
    
    print(f"Required irradiance level is: {irradiance} kWh/m2/day")
    print(f"Required land area is: {land} km2")
    
    return irradiance


def build_potential_layer(shapefile_path: str, 
                          template_raster: str, 
                          output_temp: str, 
                          output_file: str, 
                          irradiance: float, 
                          value_field: str = "ann_latilt"):
    
    # read shapefile into a geodataframe
    gdf = gpd.read_file(shapefile_path)
    
    # open the template raster and extract metadata and land mask
    with rasterio.open(template_raster) as template:

        metadata = template.meta.copy()

        # update raster data type
        metadata.update(dtype=np.float64)

        # extract land mask
        land_mask = template.read(1)
        land_mask = np.where(land_mask == 0, np.nan, 1)

        # reproject solar data to GRIDCERF standard
        gdf = gdf.to_crs(template.crs)

        # write output raster
        with rasterio.open(output_temp, 'w+', **metadata) as out:

            out_arr = out.read(1)

            # build shapes to rasterize from target geometry and field
            shapes = ((geom,value) for geom, value in zip(gdf.geometry, gdf[value_field]))

            # burn features
            burned = features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=out.transform)

            out.write_band(1, burned)

    # open burned raster
    with rasterio.open(output_temp) as src:

        metadata = src.meta.copy()

        arr = src.read(1)

        # calculate binary suitability
        arr = np.where(arr >= irradiance, 0, 1).astype(np.float64)

        # apply land mask
        arr *= land_mask

        with rasterio.open(output_file, 'w', **metadata) as dest:
            dest.write(arr, 1)

            
def construct_solar_suitability(input_potential_raster: str, 
                                output_suitability_dir: str,
                                suitability_file_list: list,
                                technology: str):
    
    # open input wind potential raster
    with rasterio.open(input_potential_raster) as solar_potential_raster:

        # update metadata datatype to int16
        metadata = solar_potential_raster.meta.copy()
        metadata.update({'dtype': rasterio.int16})

        # get array of potential suitability
        solar_suitability_array = solar_potential_raster.read(1)

    # apply suitability rasters
    for idx, i in enumerate(suitability_file_list):
        with rasterio.open(i) as src:
            solar_suitability_array += src.read(1)

    # reclassify to 0 (suitable), 1 (unsuitable)
    solar_suitability_array = np.where(solar_suitability_array == 0, 0, 1)

    # write aggregate tech-specific suitability layer to file
    final_raster = os.path.join(output_suitability_dir, f'gridcerf_nrel_solar_{technology}_centralized.tif')
    with rasterio.open(final_raster, 'w', **metadata) as dest:
        dest.write(solar_suitability_array.astype(rasterio.int16), 1)
        
    print(f"Generated solar suitability raster:  {final_raster}")
        
    return final_raster


### 4.2 Generate solar suitability rasters

In [5]:
# process each technology
for technology in ("pv", "csp"):
    
    print(f"Processing: {technology}")

    # unpack technology specific data
    slope_percent = technology_data[technology]["slope_percent"]
    unit_size_dc = technology_data[technology]["unit_size_dc"]    
    capacity_factor_ac = technology_data[technology]["capacity_factor_ac"]    
    area_percentage = technology_data[technology]["area_percentage"] 
    input_shapefile = technology_data[technology]["input_shapefile"] 
    shapefile_field = technology_data[technology]["shapefile_field"] 

    # construct technology-specific output file name
    output_file = os.path.join(technology_specific_dir, f"gridcerf_nrel_solar_{technology}_centralized_potential.tif")
    
    # calculate the irradiance threshold
    irradiance_threshold = calc_irradiance_threshold(unit_size_dc=unit_size_dc, 
                                                     capacity_factor_ac=capacity_factor_ac, 
                                                     area_percentage=area_percentage)

    # construct technical potential suitability raster
    build_potential_layer(shapefile_path=input_shapefile, 
                          template_raster=template_raster, 
                          output_temp=temp_output_raster, 
                          output_file=output_file, 
                          irradiance=irradiance_threshold, 
                          value_field=shapefile_field)
    
    # list of technology specific files to use as exclusion areas
    technology_specific_list = [os.path.join(technology_specific_dir, f"gridcerf_srtm_slope_{slope_percent}pct_or_less.tif")]

    # list of suitability files to apply to the slope raster
    suitability_file_list = common_raster_list + technology_specific_list

    # create combined suitability raster
    f = construct_solar_suitability(input_potential_raster=output_file, 
                                    output_suitability_dir=compiled_dir,
                                    suitability_file_list=suitability_file_list,
                                    technology=technology)


Processing: pv
Required irradiance level is: 4.246 kWh/m2/day
Required land area is: 1.81 km2
Generated solar suitability raster:  /Users/d3y010/repos/metarepos/vernon-etal_2022_scidata/data/gridcerf/compiled/gridcerf_solar_pv_centralized.tif
Processing: csp
Required irradiance level is: 4.615 kWh/m2/day
Required land area is: 2.41 km2
Generated solar suitability raster:  /Users/d3y010/repos/metarepos/vernon-etal_2022_scidata/data/gridcerf/compiled/gridcerf_solar_csp_centralized.tif
